## Read in Data

In [1]:
import pandas as pd
import numpy as np
import math
import copy
import itertools

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

pd.set_option('display.max_columns', None)

In [2]:
link = ('https://raw.githubusercontent.com/'
        'propublica/compas-analysis/master/compas-scores-two-years.csv')
feature = ['age', 'c_charge_degree', 'race', 'v_score_text', 'sex', 'priors_count', 
           'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid','c_jail_in', 'c_jail_out']
raw = pd.read_csv(link)[feature]
raw.head()

,age,c_charge_degree,race,v_score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Low,Male,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20
1,34,F,African-American,Low,Male,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,24,F,African-American,Low,Male,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
3,23,F,African-American,Medium,Male,1,NaN,8,0,0,NaN,NaN
4,43,F,Other,Low,Male,2,NaN,1,0,0,NaN,NaN


## Data Cleaning and Transformation

· **Race**: Caucasian (2087) and African-American (3247)

In [3]:
#basic cleaning according to the data page

df = raw[(raw.race == 'Caucasian') | (raw.race == 'African-American')]
df = df[(df.days_b_screening_arrest < 30) & (df.days_b_screening_arrest >= -30)]
df.drop(columns=['days_b_screening_arrest'], inplace = True)
df = df[df.is_recid != -1]
df = df[df.c_charge_degree != 'O']
df['length_of_stay'] = (pd.to_datetime(df["c_jail_out"]) - pd.to_datetime(df['c_jail_in'])).dt.days
df.drop(columns=['c_jail_in', 'c_jail_out'], inplace=True)

df.head()

,age,c_charge_degree,race,v_score_text,sex,priors_count,decile_score,is_recid,two_year_recid,length_of_stay
1,34,F,African-American,Low,Male,0,3,1,1,10
2,24,F,African-American,Low,Male,4,4,1,1,1
6,41,F,Caucasian,Low,Male,14,6,1,1,6
8,39,M,Caucasian,Low,Female,0,1,0,0,2
10,27,F,Caucasian,Low,Male,0,4,0,0,1


· **Age**: age < 25, 25 < age < 45, age > 45  
· **Charge Degree**: Misdemeanor or Felony  
· **Gender**: Male or Female  
· **Prior Counts**: 0, 1-3, larger than 3  
· **Length of Stay**: <= 1 week, <= 3 months or > 3 months  

In [4]:
## Encoding

catgorical = df.columns[df.dtypes == 'object']
for fea in catgorical:
    new_fea = LabelEncoder().fit_transform(df[fea])
    df[fea] = new_fea
    
df['age_cat'] = pd.cut(df['age'], bins = [0, 25, 45, 100], labels = [0, 1, 2])
df['priors_count_cat'] = pd.cut(df['priors_count'], bins = [-5, 0, 3, np.inf], labels = [0, 1, 2])
df['length_cat'] = pd.cut(df['length_of_stay'], bins = [0, 7, 90, np.inf], labels = [0, 1, 2])
df_clean = df[['race', 'age_cat', 'c_charge_degree', 'sex', 'priors_count_cat', 'length_cat', 'two_year_recid']]
df_clean = df_clean.dropna()
df_clean.head()

# to reproduce the origin experiment in paper A7, we select the age, charge_degree, sex, priors_count
# and length_of_stay as our main features

,race,age_cat,c_charge_degree,sex,priors_count_cat,length_cat,two_year_recid
1,0,1,0,1,0,1,1
2,0,0,0,1,2,0,1
6,1,1,0,1,2,0,1
8,1,1,1,0,0,0,0
10,1,1,0,1,0,0,0


Split the data as training : validation : testing = 5:1:1

In [5]:
# train_test_split

y = df_clean.two_year_recid
X = df_clean.drop(columns=['two_year_recid'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, stratify=y, random_state=123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/6, 
                                                   stratify=y_train, random_state = 123)

In [6]:
# Race as protected attributes(senesitive features)

X_train_sen = np.array(X_train['race'])
X_train = np.array(X_train.drop(columns = ['race']))
X_val_sen = np.array(X_val['race'])
X_val = np.array(X_val.drop(columns = ['race']))

# For calculating calibration, split the test dataset to different race
X_test_AA = np.array(X_test[X_test.race == 0])
X_test_CA = np.array(X_test[X_test.race == 1])

y_train = np.array(y_train)
y_val = np.array(y_val)
y_test_AA = np.array(y_test[X_test.race == 0])
y_test_CA = np.array(y_test[X_test.race == 1])

X_test = np.array(X_test)
y_test = np.array(y_test)

## A7:Information Theoretic Measures for Fairness-aware Feature Selection 

In [7]:
# shapley value calculation functions

def uniq_vals_in_arr(arr):

    uniq_vals = []
    for id_col in range(arr.shape[1]):
        uniq_vals.append(np.unique(arr[:, id_col]).tolist())
    return uniq_vals


def powerset(seq):

    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item


def info_coef(left, right):
    # Both arrays NEED same number of rows
    assert left.shape[0] == right.shape[0]
    num_rows = left.shape[0]
    num_left_cols = left.shape[1]
        
    concat_mat = np.concatenate((left, right), axis=1)
    concat_uniq_vals = uniq_vals_in_arr(concat_mat)
    concat_combos = list(itertools.product(*concat_uniq_vals))
    p_sum = 0
    for vec in concat_combos:
        p_r1_r2 = len(np.where((concat_mat == vec).all(axis=1))[0]) / num_rows
        p_r1 = len(np.where((left == vec[:num_left_cols]).all(axis=1))[0]) / num_rows
        p_r2 = len(np.where((right == vec[num_left_cols:]).all(axis=1))[0]) / num_rows
        
        if p_r1_r2 == 0 or p_r1 == 0 or p_r2 == 0:
            p_iter = 0
        else:
            p_iter = p_r1_r2 * np.log(p_r1_r2 / p_r1) / p_r1
        p_sum += np.abs(p_iter)
    return p_sum


def conditional_info_coef(left, right, conditional): 
    assert (left.shape[0] == right.shape[0]) and (left.shape[0] == conditional.shape[0])
    num_rows = left.shape[0]
    num_left_cols = left.shape[1]
    num_right_cols = right.shape[1]

    right_concat_mat = np.concatenate((right, conditional), axis=1)    
    concat_mat = np.concatenate((left, right_concat_mat), axis=1)
    concat_uniq_vals = uniq_vals_in_arr(concat_mat)
    concat_combos = list(itertools.product(*concat_uniq_vals))
    p_sum = 0
    for vec in concat_combos:
        p_r1_r2 = len(np.where((concat_mat == vec).all(axis=1))[0]) / num_rows
        p_r1 = len(np.where((left == vec[:num_left_cols]).all(axis=1))[0]) / num_rows
        p_r2 = len(np.where((concat_mat[:, num_left_cols: -num_right_cols] == vec[num_left_cols: -num_right_cols]).all(axis=1))[0]) / num_rows
        
        try:
            p_r1_given_r3 = len(np.where((concat_mat[:, :num_left_cols] == vec[:num_left_cols]).all(axis=1) & (concat_mat[:, -num_right_cols:] == vec[-num_right_cols:]).all(axis=1))[0]) / len(np.where((concat_mat[:, -num_right_cols:] == vec[-num_right_cols:]).all(axis=1))[0])
        except ZeroDivisionError:
            p_r1_given_r3 = 0
        
        if p_r1_r2 == 0 or p_r1 == 0 or p_r2 == 0 or p_r1_given_r3 == 0:
            p_iter = 0
        else:
            p_iter = p_r1_r2 * np.log(p_r1_r2 / p_r2) / p_r1_given_r3
        p_sum += np.abs(p_iter)
    return p_sum


def acc_coef(y, x_s, x_s_c, sensitive):
    conditional = np.concatenate((x_s_c, sensitive), axis=1)
    return conditional_info_coef(y, x_s, conditional)


def disc_coef(y, x_s, sensitive):
    x_s_a = np.concatenate((x_s, sensitive), axis=1)
    return info_coef(y, x_s_a) * info_coef(x_s, sensitive) * conditional_info_coef(x_s, sensitive, y)


def shapley_acc(y, x, sensitive, i):
    num_features = x.shape[1]
    lst_idx = list(range(num_features))
    lst_idx.pop(i)
    power_set = [x for x in powerset(lst_idx) if len(x) > 0]
    
    shapley = 0
    for set_idx in power_set:
        coef = math.factorial(len(set_idx)) * math.factorial(num_features - len(set_idx) - 1) / math.factorial(num_features)
        
        # Calculate v(T U {i})
        idx_xs_incl = copy.copy(set_idx)
        idx_xs_incl.append(i)
        idx_xsc_incl = list(set(list(range(num_features))).difference(set(idx_xs_incl)))
        acc_incl = acc_coef(y.reshape(-1, 1), x[:, idx_xs_incl], x[:, idx_xsc_incl], sensitive.reshape(-1, 1))
        
        # Calculate v(T)
        idx_xsc_excl = list(range(num_features))
        idx_xsc_excl.pop(i)
        idx_xsc_excl = list(set(idx_xsc_excl).difference(set(set_idx)))
        acc_excl = acc_coef(y.reshape(-1, 1), x[:, set_idx], x[:, idx_xsc_excl], sensitive.reshape(-1, 1))
        
        marginal = acc_incl - acc_excl
        shapley = shapley + coef * marginal
    return shapley


def shapley_disc(y, x, sensitive, i):
  
    num_features = x.shape[1]
    lst_idx = list(range(num_features))
    lst_idx.pop(i)
    power_set = [x for x in powerset(lst_idx) if len(x) > 0]
    
    shapley = 0
    for set_idx in power_set:
        coef = math.factorial(len(set_idx)) * math.factorial(num_features - len(set_idx) - 1) / math.factorial(num_features)
        
        # Calculate v_D(T U {i})
        idx_xs_incl = copy.copy(set_idx)
        idx_xs_incl.append(i)
        disc_incl = disc_coef(y.reshape(-1, 1), x[:, idx_xs_incl], sensitive.reshape(-1, 1))
        
        # Calculate v_D(T)
        disc_excl = disc_coef(y.reshape(-1, 1), x[:, set_idx], sensitive.reshape(-1, 1))
        
        marginal = disc_incl - disc_excl
        shapley = shapley + coef * marginal
    return shapley


def shapley(train_set):
    
    shap_acc = []
    shap_disc = []
    for i in range(5):
        acc_i = shapley_acc(train_set[0],train_set[1], train_set[2],i)
        disc_i = shapley_disc(train_set[0],train_set[1], train_set[2], i)

        shap_acc.append(acc_i)
        shap_disc.append(disc_i)
    return [shap_acc, shap_disc]


In [8]:
## Feature Selection Result

train_set = [y_train, X_train, X_train_sen]
shaply = shapley(train_set)
feature_names = ['Age', 'Charge Degree', 'Gender', 'Prior Counts', 'Length of Stay']
pd.DataFrame(list(zip(feature_names, shaply[0], shaply[1])), columns = ['Features', 'Accuracy', 'Discrimination'])

,Features,Accuracy,Discrimination
0,Age,1.210114,46772.615106
1,Charge Degree,1.087360,37525.470930
2,Gender,0.969253,37630.375782
3,Prior Counts,1.248108,47161.541173
4,Length of Stay,1.190087,46187.281878


**Conclusion**: We Applied the measure of accuracy and discrimination to the dataset. The result shows that **Age** and **Prior Counts** shows strongest proxies for discrimination. This is in line with the conclusion of paper A7. And thus we can remove **Age** to get a more fair model. To test this result, we will build two classification models based on logistic regression. The first will utilize all 5 features and the second will remove **Age** and compare their accuracy and calibration. 

## Modeling

#### With all 5 features

In [9]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

In [12]:
y_test_predict_all = lr.predict(np.delete(X_test, 0, axis = 1))
print(f'AUC:{roc_auc_score(y_test, y_test_predict_all):.2f}')
print(f'Precision:{precision_score(y_test, y_test_predict_all):.2f}')
print(f'Recall:{recall_score(y_test, y_test_predict_all):.2f}')
print(f'Accuracy:{accuracy_score(y_test, y_test_predict_all):.5f}')

AUC:0.68
Precision:0.69
Recall:0.69
Accuracy:0.67967


In [21]:
y_test_predict_AA_all = lr.predict(np.delete(X_test_AA, 0, axis = 1))
y_test_predict_CA_all = lr.predict(np.delete(X_test_CA, 0, axis = 1))
accuracy_AA_all = accuracy_score(y_test_AA, y_test_predict_AA_all)
accuracy_CA_all = accuracy_score(y_test_CA, y_test_predict_CA_all)
print(f'Calibration:{abs(accuracy_AA_all-accuracy_CA_all):.5f}')

Calibration:0.07084


#### Delete Age

In [22]:
lr2 = LogisticRegression()
lr2.fit(np.delete(X_train, 0, axis = 1), y_train)

LogisticRegression()

In [23]:
y_test_predict = lr2.predict(np.delete(X_test, [0, 1], axis = 1))
print(f'AUC:{roc_auc_score(y_test, y_test_predict):.2f}')
print(f'Precision:{precision_score(y_test, y_test_predict):.2f}')
print(f'Recall:{recall_score(y_test, y_test_predict):.2f}')
print(f'Accuracy:{accuracy_score(y_test, y_test_predict):.5f}')

AUC:0.60
Precision:0.60
Recall:0.69
Accuracy:0.59959


In [24]:
y_test_predict_AA = lr2.predict(np.delete(X_test_AA, [0, 1], axis = 1))
y_test_predict_CA = lr2.predict(np.delete(X_test_CA, [0, 1], axis = 1))
accuracy_AA = accuracy_score(y_test_AA, y_test_predict_AA)
accuracy_CA = accuracy_score(y_test_CA, y_test_predict_CA)
print(f'Calibration:{abs(accuracy_AA-accuracy_CA):.5f}')

Calibration:0.09747


#### Result

In [26]:
pd.DataFrame([['complete model', accuracy_score(y_test, y_test_predict_all), abs(accuracy_AA_all-accuracy_CA_all)], ['without age', accuracy_score(y_test, y_test_predict), abs(accuracy_AA-accuracy_CA)]], 
             columns = ['Model', 'Accuracy', 'Calibration'])

,Model,Accuracy,Calibration
0,complete model,0.679671,0.070842
1,without age,0.599589,0.097473


**Conclusion**: We can see that the removal of age may decrease the overall accuracy of the model, but increase the calibration, which is in line with our previous conclusion.